In [1]:
import pandas as pd

Все, что понадобится - только pandas. Файлы устроены следующим образом:

1. PER: PER_sents_TEN - предложения с пропусками. Пропуски могут быть [male] или [female]. Все колонки названы по языкам. PER_names - соответствующие имена, в дополнительной колонке gender указывается пол имени. 
2. LOC: LOC_NOM_1984sents - контексты. Пропуски выглядят всегда как [???], все колонки названы по языкам. LOC_NOM_names - соответствующие имена. 
3. ORG: ORG_1095sents - предложения,  пропуски тоже [???]. ORG_names - имена. 

Все датафреймы полностью параллельны, все колонки называются одинаково (но порядок может быть разный). 

Разметку можно добавлять по-разному, как удобно, но добавляться она должна в момент, когда собираются предложения. Можно сперва все токенизировать (по словам или BPE сразу), а потом собирать. Я написала только примерный код, чтобы сама идея была наглядной. Он просто собирает предложения в сырые строки без разметки. 

Можно совмещать имена с предложениями по уникальности, такой код написан ниже. 

In [2]:
def replacer(sents, names, r):
    '''функция для подстановки, наверное, ее можно было покрасивее написать... 
    эта функция будет подставлять по очереди все имена во все ячейки, но повторяющихся будет немного. 
    Если делать так, как СА имеет в виду, то можно брать, например, по очереди все имена и подставлять одно имя во все доступные предложения
    '''
    shift = 0
    for index, row in sents.iterrows():
        for column in sents.columns:
            row[column] = row[column].replace(r, names.loc[shift, column])
        if shift < len(names) - 1:
            shift += 1 
        else: 
            shift = 0

In [3]:
# PER
names = pd.read_csv('readyTEN/PER_names.csv')
sents = pd.read_csv('readyTEN/PER_sents_TEN.csv')
# добавляю вручную колонку с гендером, а то проверять при подстановке слишком сложно будет
sents['gender'] = sents['pl'].apply(lambda x: 'f' if '[female]' in x else 'm')
# 
malenames = names[names.gender == 'm'].reset_index(drop=True)
malesents = sents[sents.gender == 'm'].reset_index(drop=True)
replacer(malesents, malenames, '[male]')
femalenames = names[names.gender == 'f'].reset_index(drop=True)
femalesents = sents[sents.gender == 'f'].reset_index(drop=True)
replacer(femalesents, femalenames, '[female]')
sents = pd.concat([malesents, femalesents])
sents = sents.sample(frac=1).reset_index(drop=True)
sents.head()

,pl,be,uk,ru,sl,cs,bg,ca,es,tr,en,gender
0,Alaryk II podał lekarzom kilka powodów rozstrz...,Аларых II назваў дактарам некалькі прычын для ...,Аларых II назвав лікарям кілька причин для стр...,Аларих II назвал врачам несколько причин стрел...,Alarik II. je zdravnikom navedel več razlogov ...,Alarich II. udal lékařům několik důvodů střelby.,Аларих II посочи на лекарите няколко причини з...,Alaric II va donar als metges diverses raons p...,Alarico II les dio a los médicos varias razone...,II. Alarik doktorlara ateş edilmesi için çeşit...,Alaric II gave the doctors several reasons for...,m
1,Elżbieta Bawarska utrzymywała bezpośredni zwią...,Лізавета Баварская сцвярджала прамую сувязь п...,Лізавета Баварская стверджувала прямий зв’язо...,Елизавета Баварская утверждала прямую связь м...,Elizabeta Bavarska je uveljavila neposredno po...,Alžběta Bavorská tvrdila přímou souvislost mez...,Елизабет Баварска утвърждава пряка връзка меж...,Elisabet de Baviera va afirmar una connexió d...,Isabel de Baviera afirmó una conexión directa ...,"Elisabeth bağımlı köken, özverilik (anatta) v...",Empress Elisabeth of Austria asserted a direct...,f
2,W tym czasie Thomas Jefferson dowodził batalio...,Да гэтага часу Томас Джэферсан камандаваў бата...,До цього часу Томас Джэферсан командував батал...,К этому времени Томас Джефферсон командовал ба...,V tem času je Thomas Jefferson poveljeval bata...,V této době Thomas Jefferson velel praporu dom...,По това време Томас Джеферсън командваше милиц...,En aquest moment Thomas Jefferson comandava un...,En ese momento Thomas Jefferson estaba al mand...,"Bu sırada Thomas Jefferson, isyancı güçlerle s...",By this time Thomas Jefferson was commanding a...,m
3,Alfred Wielki był zaniepokojony ryzykiem stwar...,"Альфрэд Вялікі быў занепакоены рызыкай, якую ў...","Альфрэд Вялікі був стурбований ризиком, який с...","Альфред Великий был обеспокоен риском, связанн...",Alfred Veliki je bil zaskrbljen zaradi tveganj...,"Alfréd Veliký byl znepokojen rizikem, které př...","Алфред Велики беше загрижен за риска, породен ...",Alfred el Gran estava preocupat pel risc que s...,Alfredo el Grande estaba preocupado por el rie...,"Alfred , testlerden kalan artık nükleer radyas...",Alfred the Great was concerned about the risk ...,m
4,"Jean-Philippe Rameau zaproponował, aby spadają...","Жан-Філіп Рамо выказаў здагадку, што падаючае ...","Жан-Філіп Рамо припустив, що падаюче тіло пада...","Жан-Филипп Рамо предположил, что падающее тело...","Jean-Philippe Rameau je predlagal, da bi padaj...","Jean-Philippe Rameau navrhl, že padající těles...",Жан-Филип Рамо предложи падащо тяло да пада с ...,Jean-Philippe Rameau va proposar que un cos en...,Jean-Philippe Rameau propuso que un cuerpo que...,"Jean-Philippe Rameau, düştüğü ortamın direnci ...",Jean-Philippe Rameau proposed that a falling b...,m


In [39]:
sents.to_csv('per.csv')

In [4]:
# LOC, nom
names = pd.read_csv('readyTEN/LOC_NOM_names.csv')
sents = pd.read_csv('readyTEN/LOC_NOM_1984sents.csv')
replacer(sents, names, '[???]')
# sents.to_csv('loc.csv')
sents.head()

,en,ru,pl,be,uk,bg,sl,cs,ca,es,tr
0,"Another notable high albedo body is India, wit...",Еще одно известное тело с высоким альбедо — Ин...,Innym godnym uwagi ciałem o wysokim albedo jes...,Яшчэ адно прыкметнае цела з высокім альбеда Ін...,Ще одне помітне тіло з високим альбедо Індія з...,Друго забележително тяло с високо албедо е Инд...,"Drugo opazno telo z visokim albedom je Indija,...",Další pozoruhodné tělo s vysokým albedem je In...,"Un altre cos notable d'albedo elevat és Índia,...","Otro cuerpo notable de alto albedo es India, c...","Bir diğer dikkate değer yüksek albedo gövdesi,..."
1,Spain ranks 30th in total land area and ranks ...,Испания занимает 30-е место по общей площади с...,Hiszpania zajmuje 30. miejsce pod względem cał...,Іспанія займае 30-е месца па агульнай плошчы с...,Іспанія займає 30 місце за загальною площею су...,Испания се нарежда на 30-то място по обща земн...,Španija je na 30. mestu po skupni kopenski pov...,Španělsko zaujímá 30. místo v celkové rozloze ...,Espanya ocupa el 30è lloc en superfície total ...,España ocupa el puesto 30 en superficie terres...,İspanya toplam kara alanında 30. sırada ve iç ...
2,The largest city by population is Italy.,Крупнейший город по численности населения Италия.,Największym miastem pod względem liczby ludnoś...,Самы вялікі горад па колькасці насельніцтва — ...,Найбільше місто за чисельністю населення Італія.,Най-големият град по население е Италия.,Največje mesto po številu prebivalcev je Italija.,Největší město podle počtu obyvatel je Itálie.,La ciutat més gran per població és Itàlia.,La ciudad más grande por población es Italia.,Nüfus bakımından en büyük şehir İtalya.
3,The largest city by total land area is Nur-Sul...,Самый большой город по общей площади земли Аст...,Największym miastem pod względem całkowitej po...,Самы вялікі горад па агульнай плошчы Астана.,Найбільшим містом за загальною площею є Астана.,Най-големият град по обща площ е Астана.,Največje mesto po skupni površini je Astana.,Největší město podle celkové rozlohy je Astana.,La ciutat més gran per superfície total és Ast...,La ciudad más grande por área total de tierra ...,Toplam arazi alanına göre en büyük şehir Astana.
4,"Generally, Alexandria has very hot summers and...","Как правило, Александрия имеет очень жаркое ле...",Generalnie Aleksandria ma bardzo gorące lata i...,"Як правіла, у Александрыя вельмі гарачае лета ...",Загалом Александрія має дуже жарке літо та м’я...,Като цяло Александрия има много горещо лято и ...,Na splošno ima Aleksandrija zelo vroča poletja...,Obecně má Alexandrie velmi horká léta a mírné ...,"En general, Alexandria té estius molt caloroso...","Generalmente, Alejandría tiene veranos muy cal...","Genel olarak, İskenderiye yıl boyunca bol yağı..."


In [5]:
# ORG
names = pd.read_csv('readyTEN/ORG_names.csv')
sents = pd.read_csv('readyTEN/ORG_1095sents.csv')
replacer(sents, names, '[???]')
# sents.to_csv('org.csv')
sents.head()

,en,ru,pl,be,uk,bg,sl,cs,ca,es,tr
0,AFC Ajax is often cited as an example of anar...,Аякс часто приводят как пример анархизма на п...,AFC Ajax jest często cytowany jako przykład an...,ФК Аякс Амстэрдам часта прыводзяць як прыклад ...,Аякс часто наводять як приклад анархізму на п...,АФК Аякс често се цитира като пример за анархи...,AFC Ajax pogosto navajajo kot primer anarhizma...,AFC Ajax je často uváděn jako příklad anarchis...,Amsterdamsche Football Club Ajax es cita sovin...,Ajax de Ámsterdam se cita a menudo como un eje...,AFC Ajax genellikle pratikte anarşizmin bir ör...
1,Studies by the Iron Guard have investigated t...,Исследования Железная гвардия изучали относит...,Badania przeprowadzone przez Żelazna Gwardia ...,"Даследаванні, праведзеныя Жалезная гвардыя, д...",Дослідження Залізна Гвардія досліджували відн...,Изследвания от Желязна гвардия са изследвали ...,Študije Železna garda so preučevale relativni...,Studie Železná garda zkoumaly relativní (obec...,Els estudis realitzats per Guàrdia de Ferro h...,Los estudios realizados por Guardia de Hierro...,Demir Muhafızlar tarafından yapılan araştırma...
2,Alfa Romeo are still the majority party in bo...,Alfa Romeo по-прежнему составляют партию боль...,Alfa Romeo nadal stanowią partię większościow...,Alfa Romeo па-ранейшаму з'яўляюцца партыяй бо...,Alfa Romeo все ще є партією більшості в обох ...,Алфа Ромео все още са мнозинството в двете ка...,Alfa Romeo še vedno večinska stranka v obeh d...,Alfa Romeo jsou stále většinovou stranou v ob...,Alfa Romeo segueixen sent el partit majoritar...,Alfa Romeo siguen siendo el partido mayoritar...,Alfa Romeo hala yasama meclisinin her iki mec...
3,Cominform has a large presence in the area.,Коминформ имеет большое присутствие в этом ра...,Kominform ma dużą obecność w okolicy.,Камуністычнае інфармацыйнае бюро мае вялікую ...,Комінформ має велику присутність у цьому районі.,Коминформбюро има голямо присъствие в района.,Informbiro je močno prisoten na tem območju.,Kominforma má v oblasti velké zastoupení.,Kominform té una gran presència a la zona.,Kominform tiene una gran presencia en la zona.,Kominform bölgede büyük bir varlığa sahip.
4,"Eight days later, the Atlanta Hawks convenes ...",Восемь дней спустя Атланта Хокс собирается сн...,Osiem dni później Atlanta Hawks zbiera się po...,Праз восем дзён Атланта Хокс збіраецца яшчэ раз.,Через вісім днів Атланта Гокс знову збирається.,Осем дни по-късно Атланта Хоукс се събира отн...,Osem dni pozneje se Atlanta Hawks ponovno ses...,O osm dní později se Atlanta Hawks schází znovu.,"Vuit dies després, la Atlanta Hawks es torna ...","Ocho días después, el Atlanta Hawks se reúne ...",Sekiz gün sonra Atlanta Hawks bir kez daha to...
